# Challenge Part 2: Have customers narrow their travel searches based on temperature and precipitaion

In [39]:
import pandas as pd 
import requests 
import gmaps

from config import g_key


In [40]:
city_data_df = pd.read_csv("city_data.csv")
city_data_df

,Unnamed: 0,city,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,0,port elizabeth,-33.92,25.57,48.20,light rain,0.48,0,76,89,10.29,ZA
1,1,faanui,-16.48,-151.75,79.66,light rain,0.40,0,77,0,5.70,PF
2,2,ugra,54.78,34.33,54.64,moderate rain,1.29,0,97,100,16.53,RU
3,3,jucuapa,13.52,-88.38,91.40,light rain,0.75,0,63,64,1.92,SV
4,4,port alfred,-33.59,26.89,51.31,light rain,0.29,0,73,76,15.17,ZA
...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,jieshi,22.81,115.83,86.27,light rain,0.71,0,76,45,12.12,CN
62,62,kavaratti,10.57,72.64,83.07,light rain,0.22,0,75,100,10.74,IN
63,63,abonnema,4.71,6.79,73.99,moderate rain,1.41,0,92,100,6.87,NG
64,64,bogande,12.97,-0.14,79.38,moderate rain,1.13,0,74,100,14.67,BF


In [41]:
city_data_df.dtypes

Unnamed: 0               int64
city                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Country                 object
dtype: object

In [42]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [43]:
# Enter max temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation"))
max_temp = float(input("What is the maximum temperature you would like for your vacation"))



What is the minimum temperature you would like for your vacation0
What is the maximum temperature you would like for your vacation200


In [44]:
# Ask the customer if you want it to be raining 
rain_amount = input("Do you want in to be raining? (yes/no)")

Do you want in to be raining? (yes/no)no


In [45]:
# Ask the customer if you want it to be snowing
snow_amount = input("Do you want in to be snowing? (yes/no)")

Do you want in to be snowing? (yes/no)no


In [46]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&
                                          (city_data_df["Max Temp"] >= min_temp)&
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
    
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&
                                          (city_data_df["Max Temp"] >= min_temp)&
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_amount == "yes" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&
                                          (city_data_df["Max Temp"] >= min_temp)&
                                          (city_data_df["Rain (inches)"] > 0.0) &
                                          (city_data_df["Snow (inches)"] == 0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&
                                          (city_data_df["Max Temp"] >= min_temp)&
                                          (city_data_df["Rain (inches)"] > 0.0) &
                                          (city_data_df["Snow (inches)"] > 0)]

In [47]:
filtered_cities_df.count()

Unnamed: 0             0
city                   0
Lat                    0
Lng                    0
Max Temp               0
Current Description    0
Rain (inches)          0
Snow (inches)          0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
dtype: int64

In [51]:
hotel_df = filtered_cities_df[["city", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,city,Country,Max Temp,Lat,Lng,Hotel Name


In [53]:
params = {
    "radius": 500,
    "type" : "lodging",
    "key" : g_key
}
# Iterate through
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.google.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=parse).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found...skipping...")

In [54]:
hotel_df.head()

,city,Country,Max Temp,Lat,Lng,Hotel Name


In [55]:
city_data_df.to_csv("city_data.csv")

In [56]:
vacation_df = pd.read_csv("city_data.csv")
vacation_df.head()

,Unnamed: 0,Unnamed: 0.1,city,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,0,0,port elizabeth,-33.92,25.57,48.20,light rain,0.48,0,76,89,10.29,ZA
1,1,1,faanui,-16.48,-151.75,79.66,light rain,0.40,0,77,0,5.70,PF
2,2,2,ugra,54.78,34.33,54.64,moderate rain,1.29,0,97,100,16.53,RU
3,3,3,jucuapa,13.52,-88.38,91.40,light rain,0.75,0,63,64,1.92,SV
4,4,4,port alfred,-33.59,26.89,51.31,light rain,0.29,0,73,76,15.17,ZA


In [59]:
# Using the template add the hotel marks to the heatmap 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [60]:
marker_layer = gmaps.marker_layer[locations, info_box_content=hotel_info]
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

SyntaxError: invalid syntax (<ipython-input-60-a1642ba8b19d>, line 1)